In [1]:
!pip install gradio openai sentence-transformers faiss-cpu datasets pandas nltk pymupdf langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

In [2]:
import os
import re
import tempfile
import fitz  # PyMuPDF
import pandas as pd
import numpy as np
import nltk
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import gradio as gr

nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
from google.colab import files
uploaded = files.upload()


Saving ipc_sections.csv to ipc_sections.csv


In [4]:
ipc_df = pd.read_csv("ipc_sections.csv")
ipc_df.dropna(subset=["Description"], inplace=True)

constitution_ds = load_dataset("Sharathhebbar24/Indian-Constitution", split="train")
def preprocess_data():
    constitution_data = [
        {"source": row["article_id"], "text": sent.strip()}
        for row in constitution_ds
        if "article_desc" in row and row["article_desc"]
        for sent in nltk.sent_tokenize(row["article_desc"])
        if len(sent.strip()) > 20
    ]

    ipc_data = [
        {"source": f"Section {row['Section']}", "text": sent.strip()}
        for _, row in ipc_df.iterrows()
        for sent in nltk.sent_tokenize(str(row["Description"]))
        if len(sent.strip()) > 20
    ]

    all_data = constitution_data + ipc_data
    texts = [item["text"] for item in all_data]
    sources = [item["source"] for item in all_data]
    return texts, sources

texts, sources = preprocess_data()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/244 [00:00<?, ?B/s]

Final_IC.csv:   0%|          | 0.00/443k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/454 [00:00<?, ? examples/s]

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(texts, convert_to_tensor=False, show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

import faiss
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
os.environ["OPENAI_API_KEY"] = "your-openai-api"

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a legal expert. Given the legal text below and the user's question, provide a precise and accurate legal answer:

Legal Text:
{context}

Question:
{question}

Answer:"""
)

qa_chain = LLMChain(llm=llm, prompt=prompt_template)


<ipython-input-6-3d58d01aad78>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
<ipython-input-6-3d58d01aad78>:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=prompt_template)


In [7]:
def get_best_match(question):
    question_emb = model.encode([question], convert_to_tensor=False)
    D, I = index.search(np.array(question_emb).astype("float32"), k=1)
    best_idx = I[0][0]
    return texts[best_idx], sources[best_idx]

def is_legal_question(question):
    legal_keywords = [
        "section", "act", "law", "legal", "ipc", "article", "constitution", "penal", "rights", "duty", "court",
        "crime", "criminal", "civil", "suit", "offence", "offense", "trial", "judge", "judgment", "justice",
        "bail", "warrant", "arrest", "contract", "tort", "property", "liability", "penalty", "clause",
        "code of criminal procedure", "evidence", "procedure", "appeal", "jurisdiction", "tribunal",
        "bar council", "advocate", "litigation", "enactment", "rule", "regulation", "verdict", "plaintiff",
        "defendant", "writ", "habeas corpus", "fundamental rights", "directive principles", "preamble"
    ]
    return any(word.lower() in question.lower() for word in legal_keywords)


def legal_chatbot(question):
    greetings = ["hi", "hello", "hey"]
    if question.lower() in greetings:
        return "Hey there! Welcome to ArguLex. How may I assist you?"

    if not is_legal_question(question):
        return "⚠️ This appears to be a non-legal question. Please ask something related to law, IPC, Constitution, or legal procedures."

    context, source = get_best_match(question)
    try:
        response = qa_chain.run({"context": context, "question": question})
    except Exception as e:
        response = f"Error: {e}"

    return f"✅ **Answer:** {response}\n\n📚 **Source:** {source}"

def split_into_sentences(text):
    return re.split(r'(?<=[.!?])\s+', text.strip())

def generate_report(file_path):
    try:
        # Extracting case name and formatting
        filename = os.path.basename(file_path)
        case_name = filename.replace(".pdf", "").replace("_", " ").title()
        clean_filename = filename.replace(".pdf", "").replace(" ", "_").title()

        # Opening the PDF and extracting text
        doc = fitz.open(file_path)
        full_text = "".join(page.get_text() for page in doc)

        # Extracting Date of Judgment
        date_match = re.search(r"DATE\s+OF\s+JUDGMENT:\s*([\d/]+)", full_text, re.I)
        date = date_match.group(1) if date_match else "Unknown"

        # Extracting Judges' Names
        judge_match = re.search(r"BENCH:\s*(.+?)(?:\n|BENCH|CITATION|$)", full_text, re.I)
        judges = judge_match.group(1).strip() if judge_match else "Unknown"

        # If multiple judges are found, list them properly (comma separation assumed)
        if judges != "Unknown":
            judges = ", ".join([judge.strip() for judge in judges.split(",")])

        # Splitting the full text into sentences for summary and conclusion
        sentences = split_into_sentences(full_text)
        summary = " ".join(sentences[:3]) if len(sentences) >= 3 else full_text

        # Extracting Conclusion based on certain keywords (held, ordered, etc.)
        conclusion = "Conclusion not found."
        for i, sentence in enumerate(sentences):
            if re.search(r"(conclusion|held|ordered|final decision|therefore)", sentence, re.I):
                conclusion = " ".join(sentences[i:i+2])  # Grabbing the next two sentences for more context
                break

        # Generating the report
        report = f"""🧾 Generated Case Report
🏛️ Case Name: {case_name}
📅 Date: {date}
👨‍⚖️ Judges: {judges}

🧠 Summary:
{summary}

✅ Conclusion:
{conclusion}"""

        # Saving the report to a temporary file
        temp_dir = tempfile.gettempdir()
        report_path = os.path.join(temp_dir, f"{clean_filename}.txt")
        with open(report_path, "w", encoding="utf-8") as f:
            f.write(report)

        return report, report_path, full_text
    except Exception as e:
        return f"❌ Error: {e}", None, None



In [8]:
with gr.Blocks() as app:
    gr.Markdown("## ⚖️ ArguLex - Legal AI Assistant")

    with gr.TabItem("🧠 Chatbot & PDF Analyzer"):
        public_input = gr.Textbox(label="❓ Ask Your Legal Question")
        public_output = gr.Markdown()
        public_button = gr.Button("🔍 Get Answer")
        public_button.click(legal_chatbot, inputs=public_input, outputs=public_output)

        file_input = gr.File(label="📎 Upload Court PDF", type="filepath")
        report_output = gr.Textbox(label="📋 Case Report", lines=25)
        report_file = gr.File(label="📥 Download Report", visible=False)
        analyze_button = gr.Button("🧠 Generate Report")

        pdf_query_input = gr.Textbox(label="❓ Ask Question About Uploaded PDF")
        pdf_query_output = gr.Markdown()
        pdf_query_button = gr.Button("🔍 Get PDF-Based Answer")

        pdf_text_holder = gr.Textbox(visible=False)

        def handle_upload(file):
            summary_text, generated_file_path, pdf_text = generate_report(file)
            return summary_text, gr.update(value=generated_file_path, visible=True), pdf_text

        def query_pdf_after_report(query, pdf_text):
            if not pdf_text:
                return "❌ No PDF uploaded. Please upload a PDF first."
            response = qa_chain.run({"context": pdf_text, "question": query})
            return f"✅ **Answer:** {response}"

        analyze_button.click(handle_upload, inputs=[file_input], outputs=[report_output, report_file, pdf_text_holder])
        pdf_query_button.click(query_pdf_after_report, inputs=[pdf_query_input, pdf_text_holder], outputs=pdf_query_output)

app.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7cf8f4c1ae1b57a3e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
